# Criptoanálise contra Criptografia Simétrica

## Algoritmo de Grover vs AES

In [ ]:
from qat.lang.AQASM import Program, QRoutine, CNOT, CCNOT, SWAP, H, X, CCNOT
from qat.qpus import get_default_qpu, PyLinalg
import numpy as np

# Algoritmo de Grover vs ChaCha

## Algoritmo ChaCha

In [ ]:
def add(n):
    routine = QRoutine()
    a = routine.new_wires(n)
    b = routine.new_wires(n)
    for i in range(1, n):
        routine.apply(CNOT, b[i], a[i])
    for i in range(n-2, 0, -1):
        routine.apply(CNOT, b[i], b[i+1])
    for i in range(n-1):
        routine.apply(CCNOT, a[i], b[i], b[i+1])
    for i in range(n-1, 0, -1):
        routine.apply(CNOT, b[i], a[i])
        routine.apply(CCNOT, a[i-1], b[i-1], b[i])
    for i in range(1, n-1):
        routine.apply(CNOT, b[i], b[i+1])
    for i in range(n):
        routine.apply(CNOT, b[i], a[i])
        
    return routine

show_prog = Program()
add_routine = add(3)
show_qbits = show_prog.qalloc(6)
add_routine(show_qbits)
circuit = show_prog.to_circ()
print("total number of gates: ", len(circuit.ops))
# Display quantum circuit
%qatdisplay circuit --svg

In [ ]:
def quarterround(n):
    # a e b possuem tamanho 3 qubits
    routine = QRoutine()
    a = routine.new_wires(n)
    b = routine.new_wires(n)
    add(n)
    for i in range(1, n):
        routine.apply(CNOT, b[i], a[i])
    for i in range(n-2, 0, -1):
        routine.apply(CNOT, b[i], b[i+1])
    for i in range(n-1):
        routine.apply(CCNOT, a[i], b[i], b[i+1])
    for i in range(n-1, 0, -1):
        routine.apply(CNOT, b[i], a[i])
        routine.apply(CCNOT, a[i-1], b[i-1], b[i])
    for i in range(1, n-1):
        routine.apply(CNOT, b[i], b[i+1])
    for i in range(n):
        routine.apply(CNOT, b[i], a[i])
    
    for i in range(n):
        routine.apply(CNOT, a[i], b[i])

    #RL(2, b)
    SWAP(b[2], b[1])
    SWAP(b[1], b[0])
    SWAP(b[2], b[1])
    SWAP(b[1], b[0])
    
    return routine

show_prog = Program()
quarter_routine = quarterround(3)
show_qbits = show_prog.qalloc(6)
quarter_routine(show_qbits)
circuit = show_prog.to_circ()
print("total number of gates: ", len(circuit.ops))
# Display quantum circuit
%qatdisplay circuit --svg

In [ ]:
def toy_chacha(n):
    routine = QRoutine()
    indexes = [index for index in range(2*n)]

    con = routine.new_wires(n)
    con_prime = routine.new_wires(n)
    iv0 = routine.new_wires(n)
    iv0_prime = routine.new_wires(n)
    iv1 = routine.new_wires(n)
    iv1_prime = routine.new_wires(n)
    key = routine.new_wires(n)
    key_prime = routine.new_wires(n)
    
    routine.set_ancillae(con_prime, iv0_prime, iv1_prime, key_prime)

    # Copy initial state
    for i in range(n):
        CNOT(con[i], con_prime[i])
        CNOT(iv0[i], iv0_prime[i])
        CNOT(iv1[i], iv1_prime[i])
        CNOT(key[i], key_prime[i])


    # Quarterrounds
    routine.apply(quarterround(n), con, iv0)
    routine.apply(quarterround(n), iv1, key)
    routine.apply(quarterround(n), key, con)
    routine.apply(quarterround(n), iv0, iv1)


    # Last Step
    routine.apply(add(n), con, con_prime)
    routine.apply(add(n), iv0, iv0_prime)
    routine.apply(add(n), iv1, iv1_prime)
    routine.apply(add(n), key, key_prime)

    return routine

show_prog = Program()
chacha_qrout = toy_chacha(3)
show_qbits = show_prog.qalloc(12)
chacha_qrout(show_qbits)
circuit = show_prog.to_circ()
print("total number of gates: ", len(circuit.ops))
# Display quantum circuit
%qatdisplay circuit --svg

# Hands-on: Algoritmo de Grover

## Sumário do Algoritmo de Grover

Seja $N=2^n$. Consideramos um registrador de $n$ qubits de modo que um dos elementos da base computacional é dado por $|x_0\rangle$. Definimos inicialmente uma condição inicial uniforme

$$|\psi_0\rangle =  \frac{1}{\sqrt{2^n}}\sum_{i=0}^{2^n-1} |x\rangle$$

Daí, definimos os operadores:

$$U_{f} |x\rangle|i\rangle=|x\rangle|i\oplus f(x)\rangle,$$
chamado de oráculo (codifica a função $f$ onde $f(x_0)=-1$ e $f(x)=0$ se $x\neq x_0$, onde $x_0$ é chamado elemento marcado).
O operador $G$ é
$$G = (2|\psi_0\rangle\langle \psi_0| - \mathrm{I}_N)\otimes \mathrm{I}_2, $$
chamado de operador de inversão em torno da média.

## Os passos do algoritmo

O algoritmo tem os seguintes passos (usando $n+1$ qubits)

1. Primeiro, preparar o estado inicial $|\psi_0\rangle|-\rangle$;

2. Aplicar o operador $U_f$;

3. Aplicar o operador $G$;

4. Repetir $t$ vezes os passos 2 e 3 onde

$$
t=\left\lfloor\frac{\pi}{4}\sqrt N\right\rfloor;
$$

5. Medir o primeiro registrador na base computacional.

A medição retorna o estado $x_0$ com probabilidade de aproximadamente $1 - \frac{1}{\sqrt{N}}$.

In [ ]:
def diffusion(num_qubits):
    routine = QRoutine()
    wires = routine.new_wires(num_qubits)
    ancilla = routine.new_wires(1)
    routine.set_ancillae(ancilla)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in wires:
        H(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in wires:
        X(qubit)
    # Do multi-controlled-Z gate
    X.ctrl(len(wires))(wires, ancilla)
    # Apply transformation |11..1> -> |00..0>
    for qubit in wires:
        X(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in wires:
        H(qubit)

    return routine

show_prog = Program()
G = diffusion(12)
show_qbits = show_prog.qalloc(12)
G(show_qbits)
circuit = show_prog.to_circ()
print("total number of gates: ", len(circuit.ops))
# Display quantum circuit
%qatdisplay circuit --svg

In [ ]:
def oracle(n):
    # Toy ChaCha Enc
    routine = QRoutine()
    ancilla = routine.new_wires(1)
    routine.set_ancillae(ancilla)
    
    con = routine.new_wires(n)
    #con_prime = routine.new_wires(n)
    iv0 = routine.new_wires(n)
    #iv0_prime = routine.new_wires(n)
    iv1 = routine.new_wires(n)
    #iv1_prime = routine.new_wires(n)
    key = routine.new_wires(n)
    #key_prime = routine.new_wires(n)

    wires = [con, iv0, iv1, key]

    
    encryption = toy_chacha(n)
    with routine.compute():
        routine.apply(encryption, wires)

    # Compare Key Stream
    X.ctrl(4*n)([con, iv0, iv1, key], ancilla)
    
    # Toy ChaCha Uncompute
    routine.uncompute()

    return routine

show_prog = Program()
Uf = oracle(3)
show_qbits = show_prog.qalloc(12)
Uf(show_qbits)
circuit = show_prog.to_circ()
print("total number of gates: ", len(circuit.ops))
# Display quantum circuit
%qatdisplay circuit --svg

In [ ]:
def initial_state(num_qubits):
    routine = QRoutine()
    wires = routine.new_wires(num_qubits)
    ancilla = routine.new_wires(1)
    routine.set_ancillae(ancilla)
    for q in wires:
        H(q)
    X(ancilla)
    H(ancilla)

    return routine


show_prog = Program()
init_rout = initial_state(12)
show_qbits = show_prog.qalloc(12)
init_rout(show_qbits)
circuit = show_prog.to_circ()
print("total number of gates: ", len(circuit.ops))
# Display quantum circuit
%qatdisplay circuit --svg

In [ ]:
qprog = Program()
n = 3
num_reg = 4
num_qubits = num_reg*n
# --------------------------
# Initial state preparation
# --------------------------
con = qprog.qalloc(n)
iv0 = qprog.qalloc(n)
iv1 = qprog.qalloc(n)
key = qprog.qalloc(n)
qbits = [con,iv0,iv1,key]
cbits = qprog.calloc(num_qubits)

# Prepare initial state
#X(con[0])
qprog.apply(X,con[0])
#X(iv0[1])
qprog.apply(X,iv0[1])
#X(iv1[2])
qprog.apply(X,iv1[2])
#X(key[0])
qprog.apply(X,key[0])
#X(key[1])
qprog.apply(X,key[1])
#X(key[2])
qprog.apply(X,key[2])

initial = initial_state(num_qubits)
oracle_operator = oracle(n)
diffusion_operator = diffusion(num_qubits)

# -------------------------
# Applying Grover Algorithm
# -------------------------
initial(qbits)
for step in range(int((np.pi/4)*np.sqrt(2**num_qubits))):
    oracle_operator(qbits)
    diffusion_operator(qbits)

In [ ]:
circuit = qprog.to_circ()
print("total number of gates: ", len(circuit.ops))
# Display quantum circuit
#%qatdisplay circuit --svg

In [ ]:
# Create a job
job = circuit.to_job(nbshots=1024, qubits=qbits)
qpu = PyLinalg()

# Submit the job to the QPU
result = qpu.submit(job)

# Iterate over the final state vector to get all final components
for sample in result:
    print("State %s: probability %s +/- %s amplitude %s " % (sample.state, sample.probability, sample.err, sample.amplitude))